In [3]:
%%capture
import numpy as np
import mlflow.tensorflow
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,LSTM, Conv1D, Activation, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from tensorflow.keras.metrics import AUC
import keras


from sz_utils import data_handler
import pandas as pd

import getpass
import os

In [4]:
# check if gpu is available
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
%%capture
# collect the data
preictal, interictal = data_handler.make_patient_windows("chb01")

# make the labels
X = np.concatenate((preictal, interictal), axis=0)
y = np.concatenate((np.ones((preictal.shape[0], 1)), np.zeros((interictal.shape[0], 1))), axis=0)

# Shuffle the data
shuffle_indices = np.random.permutation(np.arange(X.shape[0]))
X = X[shuffle_indices]
y = y[shuffle_indices]

# Split the data into train and test
train_size = int(X.shape[0] * 0.8)
X_train = X[:train_size]
y_train = y[:train_size]
X_test = X[train_size:]
y_test = y[train_size:]

# Split test data into validation and test
val_size = int(X_test.shape[0] * 0.5)
X_val = X_test[:val_size]
y_val = y_test[:val_size]
X_test = X_test[val_size:]
y_test = y_test[val_size:]

In [8]:
# shapes
print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape, "X_val shape:", X_val.shape, "y_val shape:", y_val.shape, "X_test shape:", X_test.shape, "y_test shape:", y_test.shape)

X_train shape: (672, 1280, 22) y_train shape: (672, 1) X_val shape: (84, 1280, 22) y_val shape: (84, 1) X_test shape: (84, 1280, 22) y_test shape: (84, 1)


In [9]:
num_classes = 2 
input_shape_dataset = (X_train.shape[1], X_train.shape[2])
input_shape_dataset

def create_model_cnn_basic_1_layer(
    input_shape_dataset: tuple = input_shape_dataset,
    num_classes: int = num_classes,
    debug: bool = False,
    filters: int = 256,
    kernel_size: int = 3,
    pool_size: int = 2,
    dropout: float = 0.1,
    dense_size: int = 64,
    loss: str = "binary_crossentropy",
    optimizer: str = "adam",
    metrics: list = ["accuracy"],

) -> tf.keras.Model:

    """This function creates a basic convolutional neural network model with 2 convolutional layers, 2 dense layers and a softmax layer

    :param input_shape_dataset: shape of the input data
    :type input_shape_dataset: tuple
    :param num_classes: number of classes
    :type num_classes: int
    :return: return a model
    :rtype: tf.keras.Model
    """

    if debug:
        print("------------model summary---------------")
        print("input_shape_dataset", input_shape_dataset)
        print("num_classes", num_classes)

    input_shape_dataset: tuple
    
    model = Sequential()

    model.add(Conv1D(filters, kernel_size, input_shape=(input_shape_dataset)))
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Flatten())
    model.add(Dense(dense_size))
    model.add(Dropout(dropout))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    return model

In [10]:
dataset_ns = (X, y)
dataset_ns[0].shape 

(840, 1280, 22)

In [36]:
import matplotlib.pyplot as plt
from dataclasses import dataclass
import tensorflow as tf
import mlflow
from sklearn.model_selection import KFold


class Experiment:
    experiment_name: str
    model_name: str
    model: tf.keras.Model
    dataset: tuple
    hyperparameters: dict
    metrics: dict

    def __init__(self, experiment_name, model_name, model, dataset, hyperparameters, metrics, num_folds=5):
        self.experiment_name = experiment_name
        self.model_name = model_name
        self.model = model
        self.dataset = dataset
        self.hyperparameters = hyperparameters
        self.metrics = metrics
        self.num_folds = num_folds
        self.fold_no = 1
        self.acc_per_fold = []
        self.loss_per_fold = []
        self.kfold = KFold(n_splits=self.num_folds, shuffle=True)
        self.train = None
        self.test = None

    def fit_evaluate(self):
        try:
            mlflow.tensorflow.autolog()
            for i, (train, test) in enumerate(self.kfold.split(self.dataset[0], self.dataset[1])):
                with mlflow.start_run(run_name=f"fold_{i}", nested=True):
                    self.train = train
                    self.test = test
                    history = self.model.fit(self.dataset[0][train], self.dataset[1][train], epochs=self.hyperparameters["epochs"], verbose=1)
                    scores = self.model.evaluate(self.dataset[0][test], self.dataset[1][test], verbose=0)
                    print(f'Score for fold {self.fold_no}: {self.model.metrics_names[0]} of {scores[0]}; {self.model.metrics_names[1]} of {scores[1]*100}%')
                    self.acc_per_fold.append(scores[1] * 100)
                    mlflow.log_param("acc_per_fold", self.acc_per_fold)
                    self.loss_per_fold.append(scores[0])
                    self.log_metrics(history)
                    self.fold_no += 1
                # mlflow.end_run()

            mlflow.log_artifact(self._log_graphs(history), artifact_path='Artifacts')

            return history
        except Exception as e:
            print("El error se produce en el fit, el error es: ", e)

    def set_experiment(self):
        mlflow.set_experiment(self.experiment_name)

    def log_params(self):
        mlflow.log_param("model_name", self.model_name)
        for key, value in self.hyperparameters.items():
            mlflow.log_param(key, value)

    def log_metrics(self, history):
        for metric_name, metric_values in history.history.items():
            for epoch, value in enumerate(metric_values):
                mlflow.log_metric(f"{metric_name}", value, step=epoch)

    def _log_graphs(self, history):
            fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
            
            # Accuracy plot
            ax[0].plot(history.history['accuracy'], label='train')
            ax[0].plot(history.history['val_accuracy'], label='val')
            ax[0].set_title('Accuracy')
            ax[0].set_xlabel('Epoch')
            ax[0].set_ylabel('Accuracy')
            ax[0].legend()
        
            # Loss plot
            ax[1].plot(history.history['loss'], label='train')
            ax[1].plot(history.history['val_loss'], label='val')
            ax[1].set_title('Loss')
            ax[1].set_xlabel('Epoch')
            ax[1].set_ylabel('Loss')
            ax[1].legend()
            
            # Save the plots to a file
            plt.tight_layout()
            plt.savefig('graphs.png')
            return 'graphs.png'
            
    def log_artifacts(self):
        pass

    def run(self):
        self.set_experiment()
        self.log_params()
        history = self.fit_evaluate()
        artifact_path = self._log_graphs(history)
        mlflow.log_artifact(artifact_path, artifact_path='Artifacts')


# Reviewing and refactoring

In [1]:
%pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 24.4 MB/s eta 0:00:0000:0100:01
  Using cached docker-6.0.1-py3-none-any.whl (147 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
  Using cached sqlparse-0.4.3-py3-none-any.whl (42 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 33.4 MB/s eta 0:00:00a 0:00:01
  Using cached gunicorn-20.1.0-py3-none-any.whl (79 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 29.8 MB/s eta 0:00:00
  Using cached Mako-1.2.4-py3-none-any.whl (78 kB)
  Using cached PyJWT-2.6.0-py3-none-any.whl (20 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Using cached websocket_client-1.5.1-py3-none-any.whl (55 kB)
 

In [9]:
# Create a dataclass template for hyperparameters
from dataclasses import dataclass

@dataclass
class Hyperparameters:
    """Hyperparameters for the tensorflow model"""
    epochs: int
    batch_size: int
    debug: bool
    filters: int
    kernel_size: int
    pool_size: int
    dropout: float
    dense_size: int
    loss: str
    optimizer: str
    metrics: list

In [10]:
hyperparameters_baseline = Hyperparameters(
    epochs=10,
    batch_size=32,
    debug=True,
    filters=32,
    kernel_size=3,
    pool_size=2,
    dropout=0.5,
    dense_size=128,
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

hyperparameters_baseline

Hyperparameters(epochs=10, batch_size=32, debug=True, filters=32, kernel_size=3, pool_size=2, dropout=0.5, dense_size=128, loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [2]:
# Checking MLFlow
import mlflow
mlflow.set_tracking_uri("http://localhost:5000")
!tmux new-session -d -s mlflow-ui 'mlflow ui'

duplicate session: mlflow-ui


In [3]:
from sz_utils import data_handler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# check if gpu is available
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.models import Sequential



tf.config.list_physical_devices('GPU')

2023-03-29 19:35:15.320089: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 19:35:16.304869: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdirectml.d6f03b303ac3c4f2eeb8ca631688c9757b361310.so
2023-03-29 19:35:16.304938: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdxcore.so
2023-03-29 19:35:16.306556: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libd3d12.so
2023-03-29 19:35:16.661532: I tensorflow/c/logging.cc:34] DirectML device enumeration: found 1 compatible adapters.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
%%capture
# load training data

preictal, interictal = data_handler.make_patient_windows("chb01")

In [5]:
X = np.concatenate((preictal, interictal), axis=0)
y = np.concatenate((np.ones((preictal.shape[0], 1)), np.zeros((interictal.shape[0], 1))), axis=0)

# Shuffle the data
shuffle_indices = np.random.permutation(np.arange(X.shape[0]))
X = X[shuffle_indices]
y = y[shuffle_indices]

# Split the data into train and test
train_size = int(X.shape[0] * 0.8)
X_train = X[:train_size]
y_train = y[:train_size]
X_test = X[train_size:]
y_test = y[train_size:]

# Split test data into validation and test
val_size = int(X_test.shape[0] * 0.5)
X_val = X_test[:val_size]
y_val = y_test[:val_size]
X_test = X_test[val_size:]
y_test = y_test[val_size:]

# Shapes
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_val:", X_val.shape)
print("y_val:", y_val.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

X_train: (672, 1280, 22)
y_train: (672, 1)
X_val: (84, 1280, 22)
y_val: (84, 1)
X_test: (84, 1280, 22)
y_test: (84, 1)


In [8]:
# make model
model = Sequential()

model.add(Conv1D(256, 3, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64))
model.add(Dropout(0.1))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 1278, 256)         17152     
                                                                 
 activation_2 (Activation)   (None, 1278, 256)         0         
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 639, 256)         0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 163584)            0         
                                                                 
 dense_2 (Dense)             (None, 64)                10469440  
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                      

In [11]:
# track training with mlflow


# create mlflow experiment
mlflow.set_experiment("chb01")

with mlflow.start_run(run_name="auto batch size") as run:
    mlflow.tensorflow.autolog()
    model.fit(X_train, y_train, epochs=10, 
                        validation_data=(X_val, y_val))


Epoch 1/10
 5/21 [======>.......................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0115s vs `on_train_batch_end` time: 0.0180s). Check your callbacks.


21/21 [==============================] - 1s 32ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 10.6190 - val_accuracy: 0.9881
Epoch 2/10
21/21 [==============================] - 1s 30ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 10.6190 - val_accuracy: 0.9881
Epoch 3/10
21/21 [==============================] - 1s 30ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 10.6190 - val_accuracy: 0.9881
Epoch 4/10
21/21 [==============================] - 1s 31ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 10.6190 - val_accuracy: 0.9881
Epoch 5/10
21/21 [==============================] - 1s 30ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 10.6190 - val_accuracy: 0.9881
Epoch 6/10
21/21 [==============================] - 1s 29ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 10.6190 - val_accuracy: 0.9881
Epoch 7/10
21/21 [==============================] - 1s 30ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 10.6190 - val_accura

INFO:tensorflow:Assets written to: /tmp/tmpm5uyxib_/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpm5uyxib_/model/data/model/assets
2023/03/29 20:00:44 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpm5uyxib_/model, flavor: tensorflow), fall back to return ['tensorflow==2.10.0']. Set logging level to DEBUG to see the full traceback.
